# A simple Cnn experiments

In this notebook is illustrated a way to create a Cnn to classify the photos of eigth different plant species.

To do that I wrote some useful functions and objects that make to me easier the
code to read

We have:


1.   covnet module that have 
  * a basic function to create a stratified cnn with a specified hidden layers
  * basic functions to create  retrain a model specifying for how many epochs, if we want to merge  classes and if we want to delete some of them.
2. learningStages module that have learner objects that in general
  * prepare the dataset basing on the value of internal variable
  * train the model for 3 rounds where the following operation are performed
    * choose for how many epochs the model should retrain based on the state of internal variables
    * choose if the training should be done using a  class-balanced dataset
    * save the model and evaluate

3. dataExpander and dataPrep module which contain the logic to augment the dataset based on a customizable pipeline composed of multi output stages


In the following cell is contained the code that produce the submitted model
As learner I choose a SplittedLearner that in particular has the  (1.7)  Pipeline

In [1]:
from learningStages import *
from covnet import *
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import * 

hiddenLayers = [
    Dense(100,activation='relu',name='firstHidden'),
    Dense(50,activation='relu',name='summarizer'),
    Dense(100,activation='relu',name='expander')
]

#create the model to be trained by the learner
simpleCnn = simpleOnLayerCov((96,96,3),90,3,(2,2),8,hiddenLayers,)
simpleCnn.summary()

splittLearner = SplitLearner(
    simpleCnn,
    'firstSplitStage',
    baseLimit=100000,
    baseEpoch=100
)




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 96, 96, 90)        1170      
                                                                 
 batch_normalization (BatchN  (None, 96, 96, 90)       360       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 96, 96, 90)        32490     
                                                                 
 batch_normalization_1 (Batc  (None, 96, 96, 90)       360       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 90)       0         
 )                                                               
                                                        

Now we set the internal variables in particular
* set verbose to 1 to prevent the cutting of  the output coming from the fit call on model
* set the epoch range(we set to 5 to make the process a little faster than the one that I used(200))
* set limit to 10 to have a faster simulation(during my experiments it was 10e6 (choose wisely this parameter . Base the choice on the architecture where the model will be trained))

--Notes: the data expander pipeline is meant to create a new thread for each image in each stage so even if  the progress bar reach 100% the
first time you have to wait that all crated threads terminates.
Furthermore, the saving process of produced images is not parallelized, so it will be slower

With this configuration   we will have 1416800 new images to train our model on

In [6]:
splittLearner.verbose = 0
splittLearner.OnBad = False

splittLearner.baseEpoch = 10
splittLearner.epochIncrement = 1.5
splittLearner.baseLimit = 10

splittLearner.perform()


False
0:	ep:14 lim:14 equ:True


INFO:tensorflow:Assets written to: export/export/Split_firstSplitStage_1.0/assets


INFO:tensorflow:Assets written to: export/export/Split_firstSplitStage_1.0/assets


1/1 [==============================] - 0s 49ms/step - loss: 13.0828 - accuracy: 0.3333
False
1:	ep:15 lim:11 equ:True


INFO:tensorflow:Assets written to: export/export/Split_firstSplitStage_1.0/assets


INFO:tensorflow:Assets written to: export/export/Split_firstSplitStage_1.0/assets


1/1 [==============================] - 0s 42ms/step - loss: 20.5805 - accuracy: 0.0000e+00
False
2:	ep:12 lim:13 equ:True


INFO:tensorflow:Assets written to: export/export/Split_firstSplitStage_1.0/assets


INFO:tensorflow:Assets written to: export/export/Split_firstSplitStage_1.0/assets


1/1 [==============================] - 0s 39ms/step - loss: 9.9951 - accuracy: 0.0000e+00


Now visualize the result in a heatmap

In [ ]:
splittLearner.evaluate()